In [1]:
'''Need to calculate the following:
Classifier:
P(s|T) = (P(s)*P(T|s))/P(T)
T is the text of the tweet and s is the sentiment

P(T|s) = P(G|s) = product(for every g in G)(P(g|s))
Where G is the set of n-grams and g is an individual n-gram
Calculate P(g|s) using +1 for 0 values

Calculate P(T) for all T
Then add all g to sets s with counts
'''


#haven't collected corpus yet, assume this will be filled in
#Load processed tweets here
with open('hillProcessed.txt') as f:
    hillProcessed = f.read()
hillTweets = hillProcessed.split('\n')

with open('trumpProcessed.txt') as f:
    trumpProcessed = f.read()
trumpTweets = trumpProcessed.split('\n')

#Load tokens files for bigrams
with open('hill_tokens.txt') as f:
    hillTokensFile = f.read()
    
hillTokens = hillTokensFile.strip().split('\n')

with open('trump_tokens.txt') as f:
    trumpTokensFile = f.read()
    
trumpTokens = trumpTokensFile.strip().split('\n')

#Load counts of types here as hillTypes and trumpTypes
with open('hill_counts.txt') as f:
    hCountsFile = f.read()
hCounts = hCountsFile.strip().split('\n')
hillTypes = {}
for i in range (len(hCounts)):
    hCounts[i] = hCounts[i].lstrip()
    hillTypes[hCounts[i].split(' ')[1]] = int(hCounts[i].split(' ')[0]) 
    
with open('trump_counts.txt') as f:
    tCountsFile = f.read()
tCounts = tCountsFile.strip().split('\n')
trumpTypes = {}
for i in range (len(tCounts)):
    tCounts[i] = tCounts[i].lstrip()
    trumpTypes[tCounts[i].split(' ')[1]] = int(tCounts[i].split(' ')[0]) 

In [2]:
import re
import string

def runAnalysis(testFile):
    with open(testFile) as f:
        testText = f.read()
    testLines = testText.split('\n')
    testTweets = []
    
    for line in testLines:
        #remove @usernames, #hashtags, RTs, links
        result = re.sub(r"(?:@\S*|#\S*|RT|http\S*)", "", line)
        #set all letters to lowercase
        result = result.lower()
        #substitute all punctuation with a space
        result = re.sub(r"[^\w\s]",' ',result)
        result = re.sub(' +', ' ', result)
        result = result.strip()
        if result != '':
            result = "<start> " + result + " <end>"
            testTweets.append(result)
        
    totalHill = numpy.float32(0.0)
    totalTrump = numpy.float32(0.0)
    for tweet in testTweets:
        pHill, pTrump = classify(tweet)
        totalHill += pHill
        totalTrump += pTrump
        #print ("For tweet: {} \nProbability of supporting \n Hillary Clinton: {} \n Donald Trump: {}".format(tweet, pHill, pTrump))
    proHill = totalHill / (totalHill + totalTrump)
    proTrump = totalTrump / (totalHill + totalTrump)
    print("% supporting Hillary Clinton: {} \n% supporting Donald Trump: {}".format(proHill, proTrump))

In [3]:
#for every word, add it to the dictionary in this format:
#key is the word, value is another dictionary
    #within each subdictionary, have the following word as the key
    #and the number of times it follows as the value

def generateBigramsDict():
    #make a bigrams dictionary for both and each candidate
    bigrams = {}
    hillBigrams = {}
    trumpBigrams = {}
    #Add all Hillary positive bigrams to both hillBigrams and bigrams
    for i in range(len(hillTokens) - 1):
        #if the word already exists
        if hillTokens[i] in hillBigrams:
            #if the following word already exists
            if hillTokens[i+1] in hillBigrams[hillTokens[i]]:
                #add 1 to the number of occurences
                hillBigrams[hillTokens[i]][hillTokens[i+1]] += 1
                bigrams[hillTokens[i]][hillTokens[i+1]] += 1
            #add it to the subdictionary of hill
            else:
                hillBigrams[hillTokens[i]][hillTokens[i+1]] = 1
                bigrams[hillTokens[i]][hillTokens[i+1]] = 1
        #the word does not exist yet in hillBigrams
        else:
            #initialize a new subdictionary and add the first bigram
            hillBigrams[hillTokens[i]] = {hillTokens[i+1]: 1}
            bigrams[hillTokens[i]] = {hillTokens[i+1]: 1}

    #Add all Trump positive bigrams to both trumpBigrams and bigrams
    for i in range(len(trumpTokens) - 1):
        #if the word already exists
        if trumpTokens[i] in trumpBigrams:
            #if the following word already exists
            if trumpTokens[i+1] in trumpBigrams[trumpTokens[i]]:
                #add 1 to the number of occurences
                trumpBigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
                bigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
            #add it to the subdictionary of trump
            else:
                trumpBigrams[trumpTokens[i]][trumpTokens[i+1]] = 1
                #check bigrams
                if trumpTokens[i+1] in bigrams[trumpTokens[i]]:
                    bigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
                else:
                    bigrams[trumpTokens[i]][trumpTokens[i+1]] = 1
        #the word does not exist yet in trumpBigrams
        else:
            #initialize a new subdictionary and add the first bigram
            trumpBigrams[trumpTokens[i]] = {trumpTokens[i+1]: 1}
            #check bigrams
            if trumpTokens[i] in bigrams:
                #check word
                if trumpTokens[i+1] in bigrams[trumpTokens[i]]:
                    bigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
                else:
                    bigrams[trumpTokens[i]][trumpTokens[i+1]] = 1
            else:
                bigrams[trumpTokens[i]] = {trumpTokens[i+1]: 1}
    return bigrams, hillBigrams, trumpBigrams

In [4]:
from numpy import float32
from math import log, exp

'''Need to implement Laplace smoothing'''


'''MLE = count of bigram / count of first unigram
Laplace: P = (count of bigram + 1)/(count of first unigram + V)
where V is the number of possible bigrams given unigram
What do we do if the given unigram doesn't exist?'''

def smoothProb(word1, word2, sentiment):
    P = numpy.float32(0.0)
  
    #True = hillary
    if sentiment == True:
        unigramCount = 1
        #if the unigram has been seen before
        if word1 in hBigrams.keys():
            #add one for smoothing
            unigramCount = hillTypes[word1] + 1
            #if this bigram has been seen before
            if word2 in hBigrams[word1].keys():
                P = (hBigrams[word1][word2] + 1) / (unigramCount + len(hBigrams[word1]))
                #maybe log it? FIGURE OUT LATER
            #the second word has not followed the first word before
            else:
                P = 1 / (unigramCount + len(hBigrams[word1]))
        #this unigram has not been seen before
        else:
            P = unigramCount / (len(hillTokens)+len(hillTypes))
        return P
    #False = trump
    else:
        unigramCount = 1
        #if the unigram has been seen before
        if word1 in tBigrams.keys():
            #add one for smoothing
            unigramCount = trumpTypes[word1] + 1
            #if this bigram has been seen before
            if word2 in tBigrams[word1].keys():
                P = (tBigrams[word1][word2] + 1) / (unigramCount + len(tBigrams[word1]))
                #maybe log it? FIGURE OUT LATER
            #the second word has not followed the first word before
            else:
                P = 1 / (unigramCount + len(tBigrams[word1]))
        #this unigram has not been seen before
        else:
            P = unigramCount / (len(trumpTokens)+len(trumpTypes))
        return P
    
    

In [5]:
import numpy
from math import log, exp
'''Need to calculate the following:
Classifier:
P(s|T) = (P(s)*P(T|s))/P(T)
T is the text of the tweet and s is the sentiment

P(T|s) = P(G|s) = product(for every g in G)(P(g|s))
Where G is the set of n-grams and g is an individual n-gram
Calculate P(g|s) using +1 for 0 values

Calculate P(T) for all T
Then add all g to sets s with counts
'''

#probS is number of tweets in that sentiment divided by total number of tweets

def classify(tweet):
    #calculate probT
    #probT = 1/(len(hillTweets) + len(trumpTweets))
    #probS is number of tweets in that sentiment divided by total number of tweets
    probHill = len(hillTweets)/(len(hillTweets) + len(trumpTweets))
    probTrump = 1 - probHill
    probHillTweet = (probHill * classifyTweet(tweet, True))
    probTrumpTweet = (probTrump * classifyTweet(tweet, False))
    #return (probHillTweet / (probHillTweet + probTrumpTweet)), (probTrumpTweet / (probHillTweet + probTrumpTweet))
    return probHillTweet, probTrumpTweet
    
def classifyTweet(tweet, sentiment):
    tweetTokens = tweet.strip().split(' ')
    totalProb = numpy.float32(1.0)
    for i in range(0, len(tweetTokens) - 1):
        totalProb = totalProb * smoothProb(tweetTokens[i], tweetTokens[i+1], sentiment)
    return totalProb


In [7]:
#generate the bigrams dictionary
bothBigrams, hBigrams, tBigrams = generateBigramsDict()

classify('<start> make america great again <end>')
#runAnalysis('hillaryTweetsTest.txt')
runAnalysis('tweetsTest.txt')

% supporting Hillary Clinton: 0.3773236829639257 
% supporting Donald Trump: 0.6226763170360743
